# Module 2: Introduction to SQL

Data Science doesn't exist in a bubble. Organizations don't just need people to analyze data, but to store and deliver said data. In the majority of cases, data are stored in **relational databases**. To (massively) oversimplify what relational databases are, they are tables of rows and columns that contain a "key" amongst those tables that make them easy to join together. 

> A common example of this "key" could be a customer ID such as an Amazon username. Two tables could then be set up as the following:
> - Table 1: Transaction history for a business (Order #000001: user jsmith spent $50 on 1/2/2022)
> - Table 2: User information (User jsmith - John Smith, lives in Apartment 404 with phone number 867-5309)
> 
> From these two tables, we can join the tables by username to connect the username jsmith to every order they placed!

You may recall doing joins when we worked with multiple `pandas` DataFrames last semester - well, we were effectively working with databases! For this and the following module, we will be using [California School SAT Performance and Poverty Data](http://2016.padjo.org/tutorials/sqlite-data-starterpacks/) pulled from a 2016 Journalism course at Stanford. Let's do it!

In [38]:
import sqlite3

connection = sqlite3.connect("../data/cdeschools.sqlite")
cursor = connection.cursor()

Woah woah woah.. this doesn't look like when we read in a csv into a `pandas` DataFrame! What's the deal? In theory, relational databases are super simple - multiple tables with some common "stuff" between them. However, there needs to be a lot of infrastructure to make sure these data are secure, scalable, and collaborative.

> You **could** have your company's data on multiple Excel Spreadsheets on one person's laptop.. but please don't. I'm begging you. 

Having a management system for your data really pays off as a company grows, as tables can be updated by multiple people securely while ensuring that other peoples' work with the databases aren't impacted.  

The common theme thread for each of these is SQL, or **Structured Query Language**. This is a near-universal language to be able to pull (or query) specific pieces of data. 

Many companies use server-based systems to handle their databases - examples include `MySQL` and `NoSQL`. For simplicity, we will be using `sqlite` instead, which encompasses the entire database we will work with into a single file: `cdeschools.sqlite`. It definitely isn't as collaborative or secure as a full server setup, but it will allow us to write queries a **lot** faster.

Some `sqlite`-specific details:
- `sqlite3.connect()` is the function from the `sqlite3` library that loads our file, very similar to `pd.read_csv()`.
- `.cursor()` creates the actual python object that will execute the queries we create.

Let's run our first query!

In [23]:
# Select only the names of the tables in the database (there are three of them)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

output = cursor.fetchall()
print(output)

[('schools',), ('frpm',), ('satscores',)]


The meat and potatoes of what lead to our output is the string inside the `cursor.execute()` method: 

`SELECT name FROM sqlite_master WHERE type='table'`

Many queries will follow this format: `SELECT` the columns that you want `FROM` the specific table. Optionally, we can include a `WHERE` option which allows us to filter our results using some boolean expression. For example, let's try pulling the school name, street, city, and state for all **closed** schools in the `school` table.

In [29]:
cursor.execute("SELECT School, Street, City, State FROM schools WHERE StatusType == 'Closed'")
output = cursor.fetchall()
print(output[:20])

[('FAME Public Charter', '39899 Balentine Drive, Suite 335', 'Newark', 'CA'), ('Aspire California College Preparatory Academy', '2125 Jefferson Avenue', 'Berkeley', 'CA'), ('Technical, Agricultural & Nat.', '597 C Street', 'Hayward', 'CA'), ('Alameda County Opportunity', '313 West Winton Avenue', 'Hayward', 'CA'), ('Berkeley Trade & Tech. College', None, None, None), ('Canyon Valley Adult School', None, None, None), ('Roosevelt High (Defunct)', None, None, None), ('University High School (Defunc', None, None, None), ('Washington Evening High (Defun', None, None, None), ('Chabot Ranch', '2500 Fairmont Drive', 'San Leandro', 'CA'), ('Las Vistas School', '2200 Fairmont Drive', 'San Leandro', 'CA'), ('Senior Camp', '2600 Fairmont Drive', 'San Leandro', 'CA'), ('Snedigar Cottage', '2100 Fairmont Drive', 'San Leandro', 'CA'), ('Whiteford School (TMR)', '685 A Street', 'Hayward', 'CA'), ('Alameda County Special Education', '685 A Street', 'Hayward', 'CA'), ('Buena Vista School (Girls)', '2200

Awesome! We passed in our query, told the curser to `fetchall()` which sent the output of the query back to python, and allowed us to print it out.

We can also pull down every column from a particular table using a *wildcard* (*). This is shorthand for "everything that matches." It's a lot cleaner than typing out every single column name! 

In [48]:
cursor.execute("SELECT * FROM schools WHERE StatusType == 'Closed'")
output = cursor.fetchall()
print(output[:5])

[('01100170109835', '0691051', '10546', 'Closed', 'Alameda', 'Alameda County Office of Education', 'FAME Public Charter', '39899 Balentine Drive, Suite 335', '39899 Balentine Dr., Ste. 335', 'Newark', '94560-5359', 'CA', '39899 Balentine Drive, Suite 335', '39899 Balentine Dr., Ste. 335', 'Newark', '94560-5359', 'CA', None, None, None, '2005-08-29', '2015-07-31', 1, '0728', 'Directly funded', '00', 'County Office of Education (COE)', '65', 'K-12 Schools (Public)', 'TRAD', 'Traditional', 'ELEMHIGH', 'Elementary-High Combination', 'K-12', 'K-12', 'P', 0, 37.521436, -121.99391, None, None, None, None, None, None, None, None, None, '2015-09-01'), ('01100170118489', '0691051', '12283', 'Closed', 'Alameda', 'Alameda County Office of Education', 'Aspire California College Preparatory Academy', '2125 Jefferson Avenue', '2125 Jefferson Ave.', 'Berkeley', '94703-1414', 'CA', '1001 22nd Avenue, Suite 100', '1001 22nd Ave., Ste. 100', 'Oakland', '94606', 'CA', None, None, 'www.aspirepublicschools.

Awesome! This is a lot of data (even though it's only 5 rows!), and pretty hard to read in this state. From here on, we're going to convert the output into a `pandas` DataFrame before printing.

In [49]:
pd.DataFrame(output)

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,01100170109835,0691051,10546,Closed,Alameda,Alameda County Office of Education,FAME Public Charter,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,...,None,None,None,None,None,None,None,None,None,2015-09-01
1,01100170118489,0691051,12283,Closed,Alameda,Alameda County Office of Education,Aspire California College Preparatory Academy,2125 Jefferson Avenue,2125 Jefferson Ave.,Berkeley,...,None,None,None,None,None,None,None,None,None,2015-07-01
2,01100170130302,0691051,None,Closed,Alameda,Alameda County Office of Education,"Technical, Agricultural & Nat.",597 C Street,597 C St.,Hayward,...,None,None,None,None,None,None,None,None,None,1999-06-24
3,01100170130427,0691051,09265,Closed,Alameda,Alameda County Office of Education,Alameda County Opportunity,313 West Winton Avenue,313 West Winton Ave.,Hayward,...,None,None,None,None,None,None,None,None,None,2014-04-28
4,01100170131276,0691051,None,Closed,Alameda,Alameda County Office of Education,Berkeley Trade & Tech. College,None,None,None,...,None,None,None,None,None,None,None,None,None,1999-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4513,58727516118780,0642330,10457,Closed,Yuba,Wheatland,California Montessori Project,4718 Engle Road,4718 Engle Rd.,Carmichael,...,None,None,None,None,None,None,None,None,None,2009-01-30
4514,58727516118801,0642330,10458,Closed,Yuba,Wheatland,Wheatland Charter Academy,711 West Olive Street,711 West Olive St.,Wheatland,...,None,None,None,None,None,None,None,None,None,2009-01-30
4515,58727695830039,0642350,None,Closed,Yuba,Wheatland Union High,Wheatland Continuation,PO Box 398,PO Box 398,Wheatland,...,None,None,None,None,None,None,None,None,None,1999-06-24
4516,58727695830070,0642350,None,Closed,Yuba,Wheatland Union High,Wheatland Alternative Education,801 Olive Street,801 Olive St.,Wheatland,...,None,None,None,None,None,None,None,None,None,1999-06-24


## Joining Tables in SQL Queries

So far, we've learned about quite a few query *keywords*: `SELECT`, `FROM`, and `WHERE` which specify what to pull, where to pull from, and (if needed) how to filter what you want to pull. But so far, we are limited to pulling from only one table at a time. 

Recall that in a relational database setup, it's better to have smaller tables with some shared identifier between them instead of one utterly massive table with a ton of repetitive information. Then, it's easier to combine them to get exactly what you want. In SQL, this is done with the `JOIN` and `ON` keywords. We can define multiple different types of joins, depending on what data you want dropped. We went over this last semester, so here's a quick reference:

![](https://www.w3schools.com/sql/img_innerjoin.gif)
![](https://www.w3schools.com/sql/img_leftjoin.gif)
![](https://www.w3schools.com/sql/img_rightjoin.gif)
![](https://www.w3schools.com/sql/img_fulljoin.gif)

Let's walk through an example. Since we're adding more and more keywords, we are going to format things a little different instead of having the entire query on one line. Let's take a look: 

In [57]:
cursor.execute('''
    SELECT schools.School, schools.City, schools.Latitude, schools.Longitude, satscores.NumTstTakr,
        satscores.AvgScrRead, satscores.AvgScrMath, satscores.AvgScrWrite
    FROM schools
    INNER JOIN satscores
    ON schools.CDSCode = satscores.cds
    WHERE schools.StatusType == 'Closed'
''')

output = cursor.fetchall()
pd.DataFrame(output)

,0,1,2,3,4,5,6,7
0,FAME Public Charter,Newark,37.521436,-121.99391,17,503.0,546.0,505.0
1,Aspire California College Preparatory Academy,Berkeley,37.868991,-122.27844,0,NaN,NaN,NaN
2,Encinal High,Alameda,37.773616,-122.29027,132,483.0,504.0,476.0
3,None,Livermore,37.691041,-121.77055,75,516.0,523.0,515.0
4,North Campus Continuation,San Pablo,37.993898,-122.32079,22,324.0,307.0,328.0
5,Fresno Academy for Civic and Entrepreneurial L...,Fresno,36.731908,-119.79306,10,NaN,NaN,NaN
6,National University Academy - Orange Center,Fresno,36.682671,-119.78174,0,NaN,NaN,NaN
7,Northcoast Preparatory and Performing Arts Aca...,Arcata,40.863604,-124.07508,35,620.0,534.0,579.0
8,YouthBuild Charter School of California Central,Los Angeles,34.031953,-118.26627,0,NaN,NaN,NaN
9,"National University Academy, Armona",Vista,33.170564,-117.22039,0,NaN,NaN,NaN


Awesome! It's query itself is a lot to parse, so let's go step by step:
- We first `SELECT` a bunch of columns from each table. For SQL to know which column comes from which table, we do 

*table_name*.*column_name*

- `FROM` our first table, in this case the schools table.
- `INNER JOIN` our second table to get all rows that have matches in both. 
- `ON` points SQL to what to columns to search for matches. Schools in these tables are identified by CDS Codes, named CDSCode in the first table and just cds in the second. So, `schools.CDSCode = satscores.cds` lines it up for us!
- `WHERE` uses the column StatusType and only shows rows where StatusType is "Closed." 

Woo, that was a lot! In the end, we are left with a new table that has information from both of the previous two, including school name, school location, and test scores. We don't have any unneeded information either - CDS Code were useful for joining the tables together, but notice that we didn't actually need to put them in the new table.

Let's wrap things up with one last query, this time using a `LEFT` join:

In [65]:
cursor.execute('''
    SELECT s.School, sat.AvgScrMath
    FROM schools AS s
    LEFT JOIN satscores AS sat
    ON s.CDSCode = sat.cds
''')

output = cursor.fetchall()
pd.DataFrame(output)

,0,1
0,None,418.0
1,FAME Public Charter,546.0
2,Envision Academy for Arts & Technology,387.0
3,Aspire California College Preparatory Academy,NaN
4,Community School for Creative Education,NaN
...,...,...
17681,Wheatland Community Day High,NaN
17682,Wheatland Continuation,NaN
17683,Wheatland Alternative Education,NaN
17684,Academy for Career Education Charter,NaN


Cool! In this example, we just combined school name with math SAT score. We also snuck in one more keyword: `AS`. When connected to the `FROM` and `JOIN` keywords where you put the tables to query from, you can use a shorter name (similar to when we do `import pandas as pd` so we don't need to write `pandas` all the time). Since it was a left join, all schools in the school table were included, including those that may not be in the satscores table. 

For the schools not in the satscores table, they just have a NaN in the column (or "Not a number"). Add that on top of there being a fair bit of missing data in the database already, and there needs to be a **lot** of cleaning before its ready to be used.

Next week, we will continue our foray into SQL, including how to use our vast knowledge of `pandas` to make our lives a lot easier! 